In [2]:
import tensorflow as tf
import numpy as np
from rnn import CaptionGenerator
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data = load_coco_data(max_train=50, pca_features=True)

# Print out all the keys and values from the data dictionary
for k, v in data.iteritems():
    if type(v) == np.ndarray:
        print k, type(v), v.shape, v.dtype
    else:
        print k, type(v), len(v)

idx_to_word <type 'list'> 1004
train_captions <type 'numpy.ndarray'> (50, 17) int32
val_captions <type 'numpy.ndarray'> (195954, 17) int32
train_image_idxs <type 'numpy.ndarray'> (50,) int32
val_features <type 'numpy.ndarray'> (40504, 512) float32
val_image_idxs <type 'numpy.ndarray'> (195954,) int32
train_features <type 'numpy.ndarray'> (82783, 512) float32
train_urls <type 'numpy.ndarray'> (82783,) |S63
val_urls <type 'numpy.ndarray'> (40504,) |S63
word_to_idx <type 'dict'> 1004


In [4]:
word_to_idx = data['word_to_idx']
idx_to_word = {i: w for w, i in word_to_idx.iteritems()}
small_data = sample_coco_minibatch(data, split='train', batch_size=100)  # get 100 train data
train_captions, train_features, urls = small_data
start_word = idx_to_word[train_captions[0,0]]
print "start word: ", start_word

start word:  <START>


In [5]:
cap_gen = CaptionGenerator(word_to_idx, batch_size= 100, dim_feature=[196, 512], dim_embed=128,
                                   dim_hidden=128, n_time_step=16, cell_type='lstm', dtype=tf.float32)


In [6]:
logits, loss = cap_gen.build_model()

In [7]:
optimizer = tf.train.AdamOptimizer(0.03).minimize(loss)